In [1]:
# fetching web pages
import requests
# getting/processing html from pages
from bs4 import BeautifulSoup
# processing text with regular expressions
import re
# dataframes and writing to csv
import pandas as pd

In [2]:
# setup variables
# base url
root='https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/'
# how many stocks are there
numstocks=30
# target tags
targettags=['h3', 'p']
# success code
success=200

In [3]:
# check
for i in range(1, numstocks+1):
    url=root+str(i)
    print (url)

https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/1
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/2
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/3
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/4
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/5
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/6
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/7
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/8
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/9
https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q

In [ ]:
print (h3)

<h3 style="text-align: justify;">1.  Amazon.com, Inc. (NASDAQ: <a href="https://www.insidermonkey.com/insider-trading/company/amazon%20com%20inc/1018724/">AMZN</a>): $3,328.23</h3>


In [14]:
print (h3.text)

29. Twilio Inc. (NYSE: TWLO): $443.49


In [19]:
def parseheader(h3):
    # print (h3)
    # convert to string
    h3text=h3.text
    # print (h3text)
    # grab digits at the front of the string (rank)
    rank=re.search('^\d*', h3text).group()
    # print ("rank:", rank )
    # grab the $ price - dollar, zero or more digits, decimal, zero or more digits
    price=re.search('\$\d*.*\d*', h3text).group().strip('\$')
    # print ("price:", price)
    # grab colon separated exchange followed by ticker
    # exchangeticker=re.sub(' ','', re.search('\([A-Z]*: [A-Z]*\)', h3text).group())
    exchangeticker=re.sub(' ','', re.search('\([A-Z]*: [A-Z]*\-*[A-Z]*\)', h3text).group())
    # print ("exchange and ticker:", exchangeticker)
    exchange, ticker=re.sub('\(|\)','', exchangeticker).split(':')
    # print ("ticker:", ticker)
    # print ("exchange:", exchange)
    # google finance compatible ticker
    gfinanceticker=ticker+":"+exchange
    # print ("Google Finance compatible ticker:", gfinanceticker)
    # grab the stock shortname (this is ugly, but regex will probably not work here)
    shortname=h3text.split('(')[0].split('.',1)[1].strip()
    # print (shortname)
    # print (rank, longticker, shortname, price)
    return (rank, gfinanceticker, shortname, price)

In [105]:
rank, gfinanceticker, shortname, price = parseheader(h3)
print (rank, gfinanceticker, shortname, price)

29 TWLO:NYSE Twilio Inc. 443.49


In [ ]:
print (p.contents)

['Number of Hedge Funds: 147 (2020Q4)', '\nNumber of Hedge Funds: 150 (2020Q3)', '\nTotal Dollar Amount of Long Hedge Fund Positions: $15.96 billion', '\nPercent of Hedge Funds with Long Positions: 16.6%', '\nFirst Quarter Return (through February 18): 24.2%', '\nPopularity Ranking (2020Q3): 8', '\nPopularity Ranking (2020Q2): 9', '\nNoteworthy Hedge Fund Shareholders: ', <a href="https://www.insidermonkey.com/hedge-fund/coatue+management/79/">Philippe Laffont</a>, ',\xa0', <a href="https://www.insidermonkey.com/hedge-fund/scge+management/555/">Christopher Lyle</a>, ', ', <a href="https://www.insidermonkey.com/hedge-fund/fisher+asset+management/11/">Fisher Asset Management</a>]


In [ ]:
print (p.text)

Number of Hedge Funds: 147 (2020Q4)
Number of Hedge Funds: 150 (2020Q3)
Total Dollar Amount of Long Hedge Fund Positions: $15.96 billion
Percent of Hedge Funds with Long Positions: 16.6%
First Quarter Return (through February 18): 24.2%
Popularity Ranking (2020Q3): 8
Popularity Ranking (2020Q2): 9
Noteworthy Hedge Fund Shareholders: Philippe Laffont, Christopher Lyle, Fisher Asset Management


In [112]:
def parseparagraph(p):
  # print (p)
  # remove <br> tags (bs4 find and find_all remove tags only)
  for brtags in p.find_all('br'):
    brtags.extract()
  # print (p)
  # convert to list of strings
  pcontents=p.contents
  # print (pcontents)
  # strip non-breaking spaces and newlines if any, search for digits surrounded by whitespace, strip whitespace(s)
  numhedgefunds2020q4 = re.search ('\s\d*\s', pcontents[0].replace(u'\xa0', '').strip('\n')).group().strip()
  # strip non-breaking spaces and newlines if any, search for digits surrounded by whitespace, strip whitespace(s)
  numhedgefunds2020q3 = re.search ('\s\d*\s', pcontents[1].replace(u'\xa0', '').strip('\n')).group().strip()
  # search $ decimal-digits decimal decimal-digits and strip out the $
  totaldollarlongpositions = re.search ('\$\d*.\d*', pcontents[2]).group().strip('$')
  # search decimal-digits decimal decimal-digits % at the end of the string, and strip out the %
  percentfundslongpositions = re.search ('\d*.\d*%$', pcontents[3]).group().strip('%')
  # search decimal-digits decimal decimal-digits % at the end of the string, and strip out the %
  latestquarterreturn = re.search ('\d*.\d*%$', pcontents[4]).group().strip('%').strip()
  # search decimal-digits at the end of the string, and strip out whitespace(s)
  popranking2020q3 = re.search ('\d*$', pcontents[5]).group().strip()
  # search decimal-digits at the end of the string, and strip out whitespace(s)
  popranking2020q2 = re.search (' \d*$', pcontents[6]).group().strip()
  # extract noteworthy hedge funds owning stock
  notehedgies=[]
  for atags in p.find_all('a', href=True):
    # this is how we get to the href link and the text string
    # print (atags['href'], atags.text)
    notehedgies.append(atags.text)
  # print (numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2, notehedgies)
  return (numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2, notehedgies)

In [113]:
numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2, notehedgies=parseparagraph(p)
print (numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2, notehedgies)

273 245 51.5 30.8 2.2 1 1 []


In [114]:
# testing with a single case
stocks={}
url=root+str(2) # TWLO:2, BRK-A: 13
page=requests.get(url)
print (page)

if (page.status_code == success):
  print('Visited URL: {}'.format(page.url))
  html = BeautifulSoup(page.content, 'html.parser')
  # extract the <h3> tag
  h3=html.body.h3
  # extract the <p> tag
  # hack for the first page
  if i==1:
    p=html.select('p')[18]
  else: p=html.body.p
  rank, gfinanceticker, shortname, price = parseheader(h3)
  numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2, notehedgies=parseparagraph(p)

  print (rank, gfinanceticker, shortname, price)
  print (numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2, notehedgies)
  stocks[gfinanceticker]={'rank': rank, 'shortname': shortname, 'price':price, 'numhedgefunds2020q4': numhedgefunds2020q4, 'numhedgefunds2020q3': numhedgefunds2020q3, 'totaldollarlongpositions': totaldollarlongpositions, 'percentfundslongpositions': percentfundslongpositions, 'latestquarterreturn': latestquarterreturn, 'popranking2020q3': popranking2020q3, 'popranking2020q2': popranking2020q2, 'notable hedge funds': notehedgies}
else:
  print ('Oops, got error: {}', format(page.status_code))
print (stocks)

<Response [200]>
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/2/
29 TWLO:NYSE Twilio Inc. 443.49
94 71 5.0 10.6 31 60 68 ['SCGE Management', 'Alkeon Capital Management']
{'TWLO:NYSE': {'rank': '29', 'shortname': 'Twilio Inc.', 'price': '443.49', 'numhedgefunds2020q4': '94', 'numhedgefunds2020q3': '71', 'totaldollarlongpositions': '5.0', 'percentfundslongpositions': '10.6', 'latestquarterreturn': '31', 'popranking2020q3': '60', 'popranking2020q2': '68', 'notable hedge funds': ['SCGE Management', 'Alkeon Capital Management']}}


In [126]:
stocks={}
# stocks

for i in range(1, numstocks+1):
  url=root+str(i)
  # print (url)
  page=requests.get(url)
  if (page.status_code == success):
    print('Visited URL: {}'.format(page.url))
    html = BeautifulSoup(page.content, 'html.parser')
    # extract the <h3> tag
    h3=html.body.h3
    # extract the <p> tag
    if i==1: # hack for the first page
      p=html.select('p')[18]
    else: p=html.body.p
    rank, gfinanceticker, shortname, price = parseheader(h3)
    numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2, notehedgies=parseparagraph(p)

    # print (rank, gfinanceticker, shortname, price)
    # print (numhedgefunds2020q4, numhedgefunds2020q3, totaldollarlongpositions, percentfundslongpositions, latestquarterreturn, popranking2020q3, popranking2020q2)
    stocks[gfinanceticker]={'rank': rank, 'shortname': shortname, 'price':price, 'numhedgefunds2020q4': numhedgefunds2020q4, 'numhedgefunds2020q3': numhedgefunds2020q3, 'totaldollarlongpositions': totaldollarlongpositions, 'percentfundslongpositions': percentfundslongpositions, 'latestquarterreturn': latestquarterreturn, 'popranking2020q3': popranking2020q3, 'popranking2020q2': popranking2020q2, 'notablehedgefunds': notehedgies}
  else:
    print ('Oops, got error: {}', format(page.status_code))
# convert to dataframe
df=pd.DataFrame.from_dict(stocks, orient='index')
print ("Done!")

Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/2/
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/3/
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/4/
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/5/
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/6/
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/7/
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-among-hedge-funds-2020-q4-rankings-920904/8/
Visited URL: https://www.insidermonkey.com/blog/30-most-popular-stocks-amo

In [127]:
print (len(stocks))

30


In [128]:
df

,rank,shortname,price,numhedgefunds2020q4,numhedgefunds2020q3,totaldollarlongpositions,percentfundslongpositions,latestquarterreturn,popranking2020q3,popranking2020q2,notablehedgefunds
CRWD:NASDAQ,30,"CrowdStrike Holdings, Inc.",239.25,92,71,7.2,10.4,12.9,59,41,"[Tiger Global Management LLC, Coatue Management]"
TWLO:NYSE,29,Twilio Inc.,443.49,94,71,5.0,10.6,31,60,68,"[SCGE Management, Alkeon Capital Management]"
FISV:NASDAQ,28,Fiserv Inc.,113.05,94,90,5.2,10.6,0.71,25,35,"[Gabriel Plotkin’s Melvin Capital Management, ..."
PINS:NYSE,27,"Pinterest, Inc.",85.99,95,80,5.84,10.7,30.5,43,158,"[Select Equity Group,, Aubrey Capital Management]"
C:NYSE,26,Citigroup Inc.,63.48,95,91,7.12,10.7,3.8,23,23,[Kahn Brothers]
NOW:NYSE,25,"ServiceNow, Inc.",575.8,96,82,6.85,10.8,4.6,36,34,"[Tiger Global Management LLC, GQG Partners]"
CRM:NYSE,24,"Salesforce.com, inc.",247.01,97,106,10.6,10.9,11,16,17,"[Fisher Asset Management, Altimeter Capital Ma..."
WFC:NYSE,23,Wells Fargo & Company,36.96,99,90,8.75,11.2,22.9,24,33,[Berkshire Hathaway]
BAC:NYSE,22,Bank of America Corporation,34.19,99,88,35.3,11.2,12.8,27,29,"[Berkshire Hathaway, Harris Associates]"
MU:NASDAQ,21,"Micron Technology, Inc.",88.54,100,79,8.14,11.3,17.8,44,37,"[Arrowstreet Capital, Appaloosa Management LP]"


In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, CRWD:NASDAQ to AMZN:NASDAQ
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   rank                       30 non-null     object
 1   shortname                  30 non-null     object
 2   price                      30 non-null     object
 3   numhedgefunds2020q4        30 non-null     object
 4   numhedgefunds2020q3        30 non-null     object
 5   totaldollarlongpositions   30 non-null     object
 6   percentfundslongpositions  30 non-null     object
 7   latestquarterreturn        30 non-null     object
 8   popranking2020q3           30 non-null     object
 9   popranking2020q2           30 non-null     object
 10  notablehedgefunds          30 non-null     object
dtypes: object(11)
memory usage: 2.8+ KB


In [130]:
df.to_csv('insidermonkeytop30.csv', index=True)

In [131]:
!cat insidermonkeytop30.csv

,rank,shortname,price,numhedgefunds2020q4,numhedgefunds2020q3,totaldollarlongpositions,percentfundslongpositions,latestquarterreturn,popranking2020q3,popranking2020q2,notablehedgefunds
CRWD:NASDAQ,30,"CrowdStrike Holdings, Inc.",239.25,92,71,7.2,10.4,12.9,59,41,"['Tiger Global Management LLC', 'Coatue Management']"
TWLO:NYSE,29,Twilio Inc.,443.49,94,71,5.0,10.6,31,60,68,"['SCGE Management', 'Alkeon Capital Management']"
FISV:NASDAQ,28,Fiserv Inc.,113.05,94,90,5.2,10.6,0.71,25,35,"['Gabriel Plotkin’s Melvin Capital Management', 'BlueSpruce Investments']"
PINS:NYSE,27,"Pinterest, Inc.",85.99,95,80,5.84,10.7,30.5,43,158,"['Select Equity Group,', 'Aubrey Capital Management']"
C:NYSE,26,Citigroup Inc.,63.48,95,91,7.12,10.7,3.8,23,23,['Kahn Brothers']
NOW:NYSE,25,"ServiceNow, Inc.",575.8,96,82,6.85,10.8,4.6,36,34,"['Tiger Global Management LLC', 'GQG Partners']"
CRM:NYSE,24,"Salesforce.com, inc.",247.01,97,106,10.6,10.9,11,16,17,"['Fisher Asset Management', 'Altimeter Capital Management']"
W